# multiple-output models
## such as a network that takes as input a series of social media posts from a single anonymous person and tries to predict attributes of that person, such as age, gender, and income level 

In [14]:
from keras import layers
from keras import Input
from keras.models import Model

In [15]:
vocabulary_size=50000
num_income_groups=10

post_input = Input(shape = (None,), dtype="int32", name='posts')
embedded_posts = layers.Embedding(256, vocabulary_size)(post_input)

In [16]:
x = layers.Conv1D(128, 5, activation='relu')(embedded_posts)
x = layers.MaxPooling1D(5)(x)
x = layers.Conv1D(256, 5, activation='relu')(x)
x = layers.Conv1D(256, 5, activation='relu')(x)
x = layers.MaxPooling1D(5)(x)
x = layers.Conv1D(256, 5, activation='relu')(x)
x = layers.Conv1D(256, 5, activation='relu')(x)
x = layers.GlobalMaxPooling1D()(x)
x = layers.Dense(128, activation='relu')(x)


In [17]:
age_prediction    = layers.Dense(1, name='age')(x)
income_prediction = layers.Dense(num_income_groups, activation ='softmax', name='income')(x)
gender_prediction = layers.Dense(1, activation ='sigmoid', name='gender')(x)


In [ ]:
model = Model(post_input,[age_prediction, income_prediction, gender_prediction])

In [13]:
#To remedy this, you can assign different levels of importance to the loss values in their contribution to the final loss. This is useful in particular if the losses’ values use different scales. For instance, the mean squared error (MSE) loss used for the age-regression task typically takes a value around 3–5, whereas the cross- entropy loss used for the gender-classification task can be as low as 0.1. In such a situa- tion, to balance the contribution of the different losses, you can assign a weight of 10 to the crossentropy loss and a weight of 0.25 to the MSE loss.

In [8]:
model.compile(optimizer="rmsprop",
              loss = {'age':"mse",
                     "income":"categorical_crossentropy",
                     "gender":"binary_crossentropy"},
             loss_weights={"age":0.25, "income":1, "gender":10})

In [6]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
posts (InputLayer)              [(None, None)]       0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, None, 50000)  12800000    posts[0][0]                      
__________________________________________________________________________________________________
conv1d (Conv1D)                 (None, None, 128)    32000128    embedding[0][0]                  
__________________________________________________________________________________________________
max_pooling1d (MaxPooling1D)    (None, None, 128)    0           conv1d[0][0]                     
______________________________________________________________________________________________

In [ ]:
#model.fit(posts, [age_targets, income_targets, gender_targets], epochs=10, batch_size=64)